<a id="item31"></a>


## Regression with Keras assignment- This part is similar to part B only the number of epochs have been changed from 50 to 100. This notebook uses the standard scaling approach (subtracting the mean and dividing by the standard deviation)


In [4]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented. 
# If you run this notebook on a different environment, e.g. your desktop, you may need to uncomment and install certain libraries.

#!pip install numpy==2.0.2
#!pip install pandas==2.2.2
#!pip install tensorflow_cpu==2.18.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.9/60.9 kB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 88.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.1
    Uninstalling numpy-2.2.1:
      Successfully uninstalled numpy-2.2.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.2/230.2 MB 2.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.7/133.7 kB 79.7 kB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 64.9 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 52.5 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 67.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.5/24.5 MB 23.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━

## Importing the required libraries

In [8]:
import pandas as pd
import numpy as np
import keras

import warnings
warnings.simplefilter('ignore', FutureWarning)

In [9]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Input

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
from sklearn.metrics import mean_squared_error

## Read the data

In [12]:
filepath='concrete_data.csv'
concrete_data = pd.read_csv(filepath)

concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


In [13]:
concrete_data.describe()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
count,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000,1030.000000
mean,281.167864,73.895825,54.188350,181.567282,6.204660,972.918932,773.580485,45.662136,35.817961
std,104.506364,86.279342,63.997004,21.354219,5.973841,77.753954,80.175980,63.169912,16.705742
min,102.000000,0.000000,0.000000,121.800000,0.000000,801.000000,594.000000,1.000000,2.330000
25%,192.375000,0.000000,0.000000,164.900000,0.000000,932.000000,730.950000,7.000000,23.710000
50%,272.900000,22.000000,0.000000,185.000000,6.400000,968.000000,779.500000,28.000000,34.445000
75%,350.000000,142.950000,118.300000,192.000000,10.200000,1029.400000,824.000000,56.000000,46.135000
max,540.000000,359.400000,200.100000,247.000000,32.200000,1145.000000,992.600000,365.000000,82.600000


In [14]:
concrete_data.isnull().sum()

Cement                0
Blast Furnace Slag    0
Fly Ash               0
Water                 0
Superplasticizer      0
Coarse Aggregate      0
Fine Aggregate        0
Age                   0
Strength              0
dtype: int64

## Separating the predictors and the target

In [15]:
concrete_data_columns = concrete_data.columns

In [16]:
predictors = concrete_data[concrete_data_columns[concrete_data_columns != 'Strength']] # all columns except Strength
target = concrete_data['Strength'] # Strength column

<a id="item2"></a>


## Normalizing the data

In [17]:
predictors_normalized = (predictors - predictors.mean()) / predictors.std()
predictors_normalized.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


## Train-test splitting of the data

In [18]:
X_train, X_test, y_train, y_test = train_test_split(predictors_normalized, target, test_size=0.30, random_state=42)

## Building a neural network with keras using 1 hidden layer of 10 nodes,  ReLu activation function, adam optimizer and the mean squared error  as the loss function.

In [19]:
def regression_model():
    # create model
    model = Sequential()
    model.add(Input(shape=(predictors.shape[1],))) # input layer with number of predictors
    model.add(Dense(10, activation='relu'))
    
    model.add(Dense(1))
    
    # compile model
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

## Model initialization

In [20]:
model = regression_model()

## Model training by fitting the model instance on the training data

In [21]:
model.fit(X_train, y_train, validation_split=0.3, epochs=100, verbose=2)

Epoch 1/100
16/16 - 24s - loss: 1680.9344 - val_loss: 1554.1417 - 24s/epoch - 1s/step
Epoch 2/100
16/16 - 0s - loss: 1669.1929 - val_loss: 1542.9814 - 263ms/epoch - 16ms/step
Epoch 3/100
16/16 - 0s - loss: 1657.6202 - val_loss: 1531.9849 - 203ms/epoch - 13ms/step
Epoch 4/100
16/16 - 0s - loss: 1646.1360 - val_loss: 1521.2054 - 219ms/epoch - 14ms/step
Epoch 5/100
16/16 - 0s - loss: 1634.8365 - val_loss: 1510.5406 - 236ms/epoch - 15ms/step
Epoch 6/100
16/16 - 0s - loss: 1623.6521 - val_loss: 1499.7999 - 231ms/epoch - 14ms/step
Epoch 7/100
16/16 - 0s - loss: 1612.3843 - val_loss: 1488.9982 - 236ms/epoch - 15ms/step
Epoch 8/100
16/16 - 0s - loss: 1600.9375 - val_loss: 1478.2024 - 260ms/epoch - 16ms/step
Epoch 9/100
16/16 - 0s - loss: 1589.1940 - val_loss: 1467.6049 - 239ms/epoch - 15ms/step
Epoch 10/100
16/16 - 0s - loss: 1577.7819 - val_loss: 1456.4951 - 247ms/epoch - 15ms/step
Epoch 11/100
16/16 - 0s - loss: 1565.5260 - val_loss: 1445.6484 - 229ms/epoch - 14ms/step
Epoch 12/100
16/16 - 0

Epoch 94/100
16/16 - 0s - loss: 359.0712 - val_loss: 323.2752 - 254ms/epoch - 16ms/step
Epoch 95/100
16/16 - 0s - loss: 353.9666 - val_loss: 318.4427 - 220ms/epoch - 14ms/step
Epoch 96/100
16/16 - 0s - loss: 349.3028 - val_loss: 313.5012 - 212ms/epoch - 13ms/step
Epoch 97/100
16/16 - 0s - loss: 344.3737 - val_loss: 308.9240 - 233ms/epoch - 15ms/step
Epoch 98/100
16/16 - 0s - loss: 339.7422 - val_loss: 304.5389 - 237ms/epoch - 15ms/step
Epoch 99/100
16/16 - 0s - loss: 335.3555 - val_loss: 299.9909 - 227ms/epoch - 14ms/step
Epoch 100/100
16/16 - 0s - loss: 330.8753 - val_loss: 295.7544 - 224ms/epoch - 14ms/step


## Calculating model predictions over the test data using model.predict()

In [22]:
predicted_strength = model.predict(X_test)

10/10 [==============================] - 0s 4ms/step


In [23]:
#predicted_strength

## Calculating mean squared error from mean_squared_error of scikit learn

In [25]:
mse = mean_squared_error(y_test, predicted_strength)

In [26]:
print('Mean squared error on the test data is: ', mse)

Mean squared error on the test data is:  326.1943213087415


## Repeating, splitting and model trainining 50 times and calculating the mse and standard deviation of MSE

In [28]:
mse_50 = []
model = regression_model()
for i in range(50):
    X_train, X_test, y_train, y_test = train_test_split(predictors, target, test_size=0.30)
    model.fit(X_train, y_train, validation_split=0.3, epochs=100, verbose=2)
    predictions = model.predict(X_test)
    mse_50.append(mean_squared_error(y_test, predictions))

Epoch 1/100
16/16 - 2s - loss: 356073.7812 - val_loss: 266757.9375 - 2s/epoch - 94ms/step
Epoch 2/100
16/16 - 0s - loss: 216095.7500 - val_loss: 154826.9375 - 223ms/epoch - 14ms/step
Epoch 3/100
16/16 - 0s - loss: 123625.1797 - val_loss: 85384.6641 - 233ms/epoch - 15ms/step
Epoch 4/100
16/16 - 0s - loss: 67882.8359 - val_loss: 45439.8633 - 245ms/epoch - 15ms/step
Epoch 5/100
16/16 - 0s - loss: 36144.2070 - val_loss: 24021.0625 - 252ms/epoch - 16ms/step
Epoch 6/100
16/16 - 0s - loss: 19341.0410 - val_loss: 12680.8398 - 229ms/epoch - 14ms/step
Epoch 7/100
16/16 - 0s - loss: 10593.4395 - val_loss: 7057.3730 - 240ms/epoch - 15ms/step
Epoch 8/100
16/16 - 0s - loss: 6287.6577 - val_loss: 4450.0415 - 226ms/epoch - 14ms/step
Epoch 9/100
16/16 - 0s - loss: 4279.9404 - val_loss: 3311.5383 - 246ms/epoch - 15ms/step
Epoch 10/100
16/16 - 0s - loss: 3382.6729 - val_loss: 2836.8801 - 214ms/epoch - 13ms/step
Epoch 11/100
16/16 - 0s - loss: 2974.0701 - val_loss: 2651.2710 - 228ms/epoch - 14ms/step
Epoc

Epoch 93/100
16/16 - 0s - loss: 594.0893 - val_loss: 701.1328 - 222ms/epoch - 14ms/step
Epoch 94/100
16/16 - 0s - loss: 587.1440 - val_loss: 696.0736 - 217ms/epoch - 14ms/step
Epoch 95/100
16/16 - 0s - loss: 581.3471 - val_loss: 689.8041 - 225ms/epoch - 14ms/step
Epoch 96/100
16/16 - 0s - loss: 575.7018 - val_loss: 682.8216 - 228ms/epoch - 14ms/step
Epoch 97/100
16/16 - 0s - loss: 569.3386 - val_loss: 679.9827 - 229ms/epoch - 14ms/step
Epoch 98/100
16/16 - 0s - loss: 563.5082 - val_loss: 671.1576 - 222ms/epoch - 14ms/step
Epoch 99/100
16/16 - 0s - loss: 556.8759 - val_loss: 666.2116 - 219ms/epoch - 14ms/step
Epoch 100/100
16/16 - 0s - loss: 552.2911 - val_loss: 659.8632 - 231ms/epoch - 14ms/step
10/10 [==============================] - 0s 4ms/step
Epoch 1/100
16/16 - 0s - loss: 610.3310 - val_loss: 554.2181 - 336ms/epoch - 21ms/step
Epoch 2/100
16/16 - 0s - loss: 602.2470 - val_loss: 548.2989 - 219ms/epoch - 14ms/step
Epoch 3/100
16/16 - 0s - loss: 594.7792 - val_loss: 540.2022 - 227ms

Epoch 86/100
16/16 - 0s - loss: 271.1409 - val_loss: 275.1264 - 206ms/epoch - 13ms/step
Epoch 87/100
16/16 - 0s - loss: 268.5618 - val_loss: 276.1733 - 223ms/epoch - 14ms/step
Epoch 88/100
16/16 - 0s - loss: 267.6508 - val_loss: 271.9563 - 221ms/epoch - 14ms/step
Epoch 89/100
16/16 - 0s - loss: 264.4592 - val_loss: 269.8404 - 215ms/epoch - 13ms/step
Epoch 90/100
16/16 - 0s - loss: 262.8750 - val_loss: 269.7769 - 222ms/epoch - 14ms/step
Epoch 91/100
16/16 - 0s - loss: 260.3240 - val_loss: 266.0014 - 219ms/epoch - 14ms/step
Epoch 92/100
16/16 - 0s - loss: 258.0715 - val_loss: 264.8857 - 226ms/epoch - 14ms/step
Epoch 93/100
16/16 - 0s - loss: 256.8008 - val_loss: 260.5221 - 219ms/epoch - 14ms/step
Epoch 94/100
16/16 - 0s - loss: 254.2404 - val_loss: 261.7433 - 224ms/epoch - 14ms/step
Epoch 95/100
16/16 - 0s - loss: 252.0210 - val_loss: 257.2201 - 225ms/epoch - 14ms/step
Epoch 96/100
16/16 - 0s - loss: 250.3541 - val_loss: 255.7628 - 217ms/epoch - 14ms/step
Epoch 97/100
16/16 - 0s - loss: 

Epoch 79/100
16/16 - 0s - loss: 155.7102 - val_loss: 146.1657 - 206ms/epoch - 13ms/step
Epoch 80/100
16/16 - 0s - loss: 153.5942 - val_loss: 144.8613 - 207ms/epoch - 13ms/step
Epoch 81/100
16/16 - 0s - loss: 154.5882 - val_loss: 143.4021 - 218ms/epoch - 14ms/step
Epoch 82/100
16/16 - 0s - loss: 152.3465 - val_loss: 142.9634 - 207ms/epoch - 13ms/step
Epoch 83/100
16/16 - 0s - loss: 151.4376 - val_loss: 142.8264 - 212ms/epoch - 13ms/step
Epoch 84/100
16/16 - 0s - loss: 150.2234 - val_loss: 142.9209 - 212ms/epoch - 13ms/step
Epoch 85/100
16/16 - 0s - loss: 150.1021 - val_loss: 142.6176 - 215ms/epoch - 13ms/step
Epoch 86/100
16/16 - 0s - loss: 149.4336 - val_loss: 139.5601 - 207ms/epoch - 13ms/step
Epoch 87/100
16/16 - 0s - loss: 150.9391 - val_loss: 143.0737 - 206ms/epoch - 13ms/step
Epoch 88/100
16/16 - 0s - loss: 149.2558 - val_loss: 139.3290 - 211ms/epoch - 13ms/step
Epoch 89/100
16/16 - 0s - loss: 147.8131 - val_loss: 139.2716 - 215ms/epoch - 13ms/step
Epoch 90/100
16/16 - 0s - loss: 

Epoch 72/100
16/16 - 0s - loss: 119.8996 - val_loss: 106.5490 - 214ms/epoch - 13ms/step
Epoch 73/100
16/16 - 0s - loss: 115.2240 - val_loss: 106.9812 - 214ms/epoch - 13ms/step
Epoch 74/100
16/16 - 0s - loss: 114.6864 - val_loss: 106.2145 - 218ms/epoch - 14ms/step
Epoch 75/100
16/16 - 0s - loss: 111.2686 - val_loss: 105.6531 - 225ms/epoch - 14ms/step
Epoch 76/100
16/16 - 0s - loss: 113.3722 - val_loss: 105.5006 - 215ms/epoch - 13ms/step
Epoch 77/100
16/16 - 0s - loss: 111.7952 - val_loss: 107.1359 - 218ms/epoch - 14ms/step
Epoch 78/100
16/16 - 0s - loss: 113.1030 - val_loss: 106.5649 - 211ms/epoch - 13ms/step
Epoch 79/100
16/16 - 0s - loss: 113.6873 - val_loss: 112.0448 - 224ms/epoch - 14ms/step
Epoch 80/100
16/16 - 0s - loss: 117.7188 - val_loss: 108.8929 - 225ms/epoch - 14ms/step
Epoch 81/100
16/16 - 0s - loss: 116.8010 - val_loss: 105.8364 - 216ms/epoch - 13ms/step
Epoch 82/100
16/16 - 0s - loss: 114.4820 - val_loss: 106.1355 - 218ms/epoch - 14ms/step
Epoch 83/100
16/16 - 0s - loss: 

...

16/16 - 0s - loss: 47.5945 - val_loss: 44.4509 - 108ms/epoch - 7ms/step
Epoch 51/100
16/16 - 0s - loss: 44.8273 - val_loss: 44.4904 - 107ms/epoch - 7ms/step
Epoch 52/100
16/16 - 0s - loss: 45.5152 - val_loss: 43.6023 - 102ms/epoch - 6ms/step
Epoch 53/100
16/16 - 0s - loss: 45.3002 - val_loss: 44.1124 - 102ms/epoch - 6ms/step
Epoch 54/100
16/16 - 0s - loss: 43.7965 - val_loss: 45.3859 - 105ms/epoch - 7ms/step
Epoch 55/100
16/16 - 0s - loss: 44.9592 - val_loss: 43.6453 - 109ms/epoch - 7ms/step
Epoch 56/100
16/16 - 0s - loss: 45.4875 - val_loss: 44.9851 - 116ms/epoch - 7ms/step
Epoch 57/100
16/16 - 0s - loss: 43.5375 - val_loss: 43.9897 - 113ms/epoch - 7ms/step
Epoch 58/100
16/16 - 0s - loss: 43.7876 - val_loss: 44.0596 - 109ms/epoch - 7ms/step
Epoch 59/100
16/16 - 0s - loss: 46.0677 - val_loss: 57.4724 - 115ms/epoch - 7ms/step
Epoch 60/100
16/16 - 0s - loss: 47.6874 - val_loss: 44.1281 - 106ms/epoch - 7ms/step
Epoch 61/100
16/16 - 0s - loss: 45.4399 - val_loss: 48.0944 - 113ms/epoch - 7m

16/16 - 0s - loss: 44.4360 - val_loss: 48.6186 - 115ms/epoch - 7ms/step
Epoch 47/100
16/16 - 0s - loss: 44.2062 - val_loss: 51.3618 - 105ms/epoch - 7ms/step
Epoch 48/100
16/16 - 0s - loss: 43.8292 - val_loss: 48.0214 - 102ms/epoch - 6ms/step
Epoch 49/100
16/16 - 0s - loss: 44.3351 - val_loss: 48.1656 - 105ms/epoch - 7ms/step
Epoch 50/100
16/16 - 0s - loss: 45.8733 - val_loss: 48.1229 - 109ms/epoch - 7ms/step
Epoch 51/100
16/16 - 0s - loss: 43.2203 - val_loss: 49.3843 - 110ms/epoch - 7ms/step
Epoch 52/100
16/16 - 0s - loss: 43.7313 - val_loss: 50.1871 - 110ms/epoch - 7ms/step
Epoch 53/100
16/16 - 0s - loss: 43.2463 - val_loss: 49.0521 - 104ms/epoch - 6ms/step
Epoch 54/100
16/16 - 0s - loss: 44.3387 - val_loss: 48.3591 - 90ms/epoch - 6ms/step
Epoch 55/100
16/16 - 0s - loss: 47.5067 - val_loss: 47.8200 - 104ms/epoch - 6ms/step
Epoch 56/100
16/16 - 0s - loss: 46.6647 - val_loss: 50.0897 - 102ms/epoch - 6ms/step
Epoch 57/100
16/16 - 0s - loss: 43.8010 - val_loss: 49.7384 - 102ms/epoch - 6ms

16/16 - 0s - loss: 44.6008 - val_loss: 53.4257 - 108ms/epoch - 7ms/step
Epoch 43/100
16/16 - 0s - loss: 47.2099 - val_loss: 55.9959 - 107ms/epoch - 7ms/step
Epoch 44/100
16/16 - 0s - loss: 43.6511 - val_loss: 55.6277 - 104ms/epoch - 6ms/step
Epoch 45/100
16/16 - 0s - loss: 43.2147 - val_loss: 53.7192 - 110ms/epoch - 7ms/step
Epoch 46/100
16/16 - 0s - loss: 45.0336 - val_loss: 54.0911 - 102ms/epoch - 6ms/step
Epoch 47/100
16/16 - 0s - loss: 45.9998 - val_loss: 53.7793 - 105ms/epoch - 7ms/step
Epoch 48/100
16/16 - 0s - loss: 48.7768 - val_loss: 54.4207 - 100ms/epoch - 6ms/step
Epoch 49/100
16/16 - 0s - loss: 45.3006 - val_loss: 53.4052 - 109ms/epoch - 7ms/step
Epoch 50/100
16/16 - 0s - loss: 46.1307 - val_loss: 54.1558 - 111ms/epoch - 7ms/step
Epoch 51/100
16/16 - 0s - loss: 47.8847 - val_loss: 57.5096 - 101ms/epoch - 6ms/step
Epoch 52/100
16/16 - 0s - loss: 43.6304 - val_loss: 53.7786 - 106ms/epoch - 7ms/step
Epoch 53/100
16/16 - 0s - loss: 45.3063 - val_loss: 54.0045 - 111ms/epoch - 7m

16/16 - 0s - loss: 47.1984 - val_loss: 43.4117 - 148ms/epoch - 9ms/step
Epoch 39/100
16/16 - 0s - loss: 46.8575 - val_loss: 44.2154 - 147ms/epoch - 9ms/step
Epoch 40/100
16/16 - 0s - loss: 46.9207 - val_loss: 43.8410 - 107ms/epoch - 7ms/step
Epoch 41/100
16/16 - 0s - loss: 46.7358 - val_loss: 42.0504 - 115ms/epoch - 7ms/step
Epoch 42/100
16/16 - 0s - loss: 45.4536 - val_loss: 42.2849 - 104ms/epoch - 6ms/step
Epoch 43/100
16/16 - 0s - loss: 45.8927 - val_loss: 42.9191 - 124ms/epoch - 8ms/step
Epoch 44/100
16/16 - 0s - loss: 46.4768 - val_loss: 42.3775 - 136ms/epoch - 9ms/step
Epoch 45/100
16/16 - 0s - loss: 45.4323 - val_loss: 41.8853 - 112ms/epoch - 7ms/step
Epoch 46/100
16/16 - 0s - loss: 46.5045 - val_loss: 45.9803 - 110ms/epoch - 7ms/step
Epoch 47/100
16/16 - 0s - loss: 48.2643 - val_loss: 53.4458 - 117ms/epoch - 7ms/step
Epoch 48/100
16/16 - 0s - loss: 48.0793 - val_loss: 42.8400 - 117ms/epoch - 7ms/step
Epoch 49/100
16/16 - 0s - loss: 45.6490 - val_loss: 47.7717 - 127ms/epoch - 8m

16/16 - 0s - loss: 48.5298 - val_loss: 44.8066 - 108ms/epoch - 7ms/step
Epoch 35/100
16/16 - 0s - loss: 50.1567 - val_loss: 39.5939 - 100ms/epoch - 6ms/step
Epoch 36/100
16/16 - 0s - loss: 49.1572 - val_loss: 41.5990 - 102ms/epoch - 6ms/step
Epoch 37/100
16/16 - 0s - loss: 49.2757 - val_loss: 39.6016 - 108ms/epoch - 7ms/step
Epoch 38/100
16/16 - 0s - loss: 47.8363 - val_loss: 42.7129 - 100ms/epoch - 6ms/step
Epoch 39/100
16/16 - 0s - loss: 48.6647 - val_loss: 39.6053 - 100ms/epoch - 6ms/step
Epoch 40/100
16/16 - 0s - loss: 48.7118 - val_loss: 40.6407 - 103ms/epoch - 6ms/step
Epoch 41/100
16/16 - 0s - loss: 52.2635 - val_loss: 43.8840 - 102ms/epoch - 6ms/step
Epoch 42/100
16/16 - 0s - loss: 50.8955 - val_loss: 40.9871 - 114ms/epoch - 7ms/step
Epoch 43/100
16/16 - 0s - loss: 50.1565 - val_loss: 41.0471 - 111ms/epoch - 7ms/step
Epoch 44/100
16/16 - 0s - loss: 48.9448 - val_loss: 40.0111 - 110ms/epoch - 7ms/step
Epoch 45/100
16/16 - 0s - loss: 48.1755 - val_loss: 41.0339 - 112ms/epoch - 7m

## Calculating mean and standard deviation of the MSEs using numpy 

In [29]:
mse_arr = np.array(mse_50)
mean_mse = np.mean(mse_arr)
std_mse = np.std(mse_arr)

In [31]:
print('The mean MSE after training the model 50 times with 100 epochs: ',mean_mse, ' and the standard deviation is: ', std_mse)

The mean MSE after training the model 50 times with 100 epochs:  83.85968388580191  and the standard deviation is:  93.42974445371604


Compared to Part B of the assignment, by increasing the epochs from 50 to 100, the performance has improved in terms of mean MSE, however, the standard deviation is more. Mean MSE and standard deviation in partB (with 50 epochs) was 92.61864332056368 and 85.06794677610809 respectively. 